In [5]:
import numpy as np
from midi_to_name import midi_to_name, hit_mapping
from mido import MidiFile, MetaMessage, tick2second

mid = MidiFile('midi_drums.mid')
ticks_per_beat = mid.ticks_per_beat
track = mid.tracks[0]
tempo = 0
hits = []
last_hit_time = 0
total_time = 0
for msg in track:
    if type(msg) == MetaMessage:
        if msg.type == 'set_tempo':

            tempo = msg.tempo

    total_time += tick2second(msg.time, ticks_per_beat, tempo)
    last_hit_time += tick2second(msg.time, ticks_per_beat, tempo)
    print(last_hit_time)
    if msg.type == 'note_on' and msg.note in midi_to_name.keys():
        hit = midi_to_name[msg.note]
        if hit in hit_mapping.keys():
            simplified_hit = hit_mapping[hit]
            hits.append({"hit": simplified_hit, "last_hit_time": last_hit_time, "total_time": total_time, "strength": msg.velocity})
            last_hit_time = 0
    
    

print(hits)


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
[{'hit': 'ride', 'last_hit_time': 0.0, 'total_time': 0.0, 'strength': 82}, {'hit': 'low_tom', 'last_hit_time': 0.0, 'total_time': 0.0, 'strength': 57}, {'hit': 'snare', 'last_hit_time': 0.0, 'total_time': 0.0, 'strength': 116}, {'hit': 'ride', 'last_hit_time': 0.0, 'total_time': 0.0, 'strength': 96}, {'hit': 'ride', 'last_hit_time': 0.0, 'total_time': 0.0, 'strength': 47}, {'hit': 'low_tom', 'last_hit_time': 0.0, 'total_time': 0.0, 'strength': 56}, {'hit': 'snare', 'last_hit_time': 0.0, 'total_time': 0.0, 'strength': 93}, {'hit': 

# MIDI File interpretation
* A MIDI file contains multiple tracks, which correspond to different instruments. For the sake of this project, we will focus only on the first track [0].
* MetaMessage: We will ignore most MetaMessages except for:
    * 'set_tempo': the 'tempo' key on this message contains the tempo in microseconds per quarter note. We can convert between BPM and this tempo using the 'mido' function tempo2bpm().
    Note: Ideally, our robot will try to keep up with the midi file tempo, but we might need to slow it down to match the robot's movements.
    * 'end_of_track': Message indicating the end of the track.
* Message: One of 'note_on' 'note_off'. Since the drums don't usually have a duration for each hit, we can ignore all 'note_off' messages.
    * 'note': Indicates what hit (snare, rim, tom, hihat, etc...) is required. Ranges from 0-127. A piano C1 is equivalent to note 36 in MIDI. We will use the following mapping: 
    ![midi_notes_map](midi_notes_map.jpg) 
    * 'velocity': Indicates the strength of the hit. Ranges from 0-127
    * 'time': Indicates the "ticks" since the last message was received. Knowing the 'ticks_per_beat' parameter, it's possible to obtain the absolute value in milliseconds, when the message is triggered. We can use function tick2second() to convert to seconds, a more useful measurement for our simulation.


In this book we will read each MIDI message and convert it to a format that's readable for our trajectory planning and control.

In [2]:
from manipulation import running_as_notebook

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    Parser,
    DiagramBuilder,
    MeshcatVisualizer,
    ModelVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Role,
    BodyIndex,
    RigidTransform,
    RotationMatrix,
    SceneGraph,
    Simulator,
    StartMeshcat,
    RollPitchYaw,
)

from IPython.display import clear_output

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
stiffness=20000
dissipation=2

drum_kit = f"""
<?xml version="1.0" ?>
<sdf version="1.7">
    <model name="drumkit">
    <static>true</static>
    <link name="snare">
    <collision name="snare">
        <pose>.1 .1 .4 0 .14 0</pose>
        <geometry>
            <cylinder>
                <length>.18</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="snare">
        <pose>.1 .1 .4 0 .14 0</pose>
        <geometry>
            <cylinder>
                <length>.18</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>
    <link name="tom1">
    <collision name="tom1">
        <pose>-.2 .3 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.1</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="tom1">
        <pose>-.2 .3 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.15</length>
                <radius>0.14</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="tom2">
    <collision name="tom2">
        <pose>-.2 .3 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.1</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="tom2">
        <pose>-.2 .62 .6 0 .55 0</pose>
        <geometry>
            <cylinder>
                <length>.15</length>
                <radius>0.16</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="tom3">
    <collision name="tom3">
        <pose>.1 .9 .3 0 0 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="tom3">
        <pose>.1 .9 .3 0 0 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.2</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="hihat">
    <collision name="hihat">
        <pose>-.1 -.1 .6 0 0 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.2 .2 .02</radii>
            </ellipsoid>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="hihat">
        <pose>.1 -.25 .6 0 0 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.2 .2 .02</radii>
            </ellipsoid>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="crash">
    <collision name="crash">
         <pose>-.4 .1 .85 0 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.26 .26 .005</radii>
            </ellipsoid>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="crash">
        <pose>-.4 .1 .85 0 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.26 .26 .005</radii>
            </ellipsoid>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="ride">
    <collision name="ride">
        <pose>-.25 1.1 .65 .3 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.3 .3 .005</radii>
            </ellipsoid>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="ride">
        <pose>-.25 1.1 .65 .3 .3 0</pose>
        <geometry>
            <ellipsoid>
                <radii>.3 .3 .005</radii>
            </ellipsoid>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    <link name="kick">
    <collision name="kick">
        <pose>-.3 .47 .25 0 1.57 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.25</radius>
            </cylinder>
        </geometry>

        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="kick">
        <pose>-.3 .47 .25 0 1.57 0</pose>
        <geometry>
            <cylinder>
                <length>.4</length>
                <radius>0.25</radius>
            </cylinder>
        </geometry>
        <material>
            <diffuse>1 0 0 1</diffuse>
        </material>
    </visual>
    </link>

    </model>
    </sdf>
"""


drum_sticks  = f"""
<?xml version="1.0" ?>
<sdf version="1.7">
    <model name="sticks">
    <link name="stick1">
    <inertial>
        <pose>0 .85 10.2 0 .2 0</pose>
        <mass>0.3</mass>
        <inertia>
            <ixx>{1/12 * 0.22**2 }</ixx>
            <iyy>{1/12 * 0.22**2 }</iyy>
            <izz>{1/2 * 0.01**2}</izz>
        </inertia>
        </inertial>
    <collision name="stick1">
        <pose>0 .85 10.2 0 .2 0</pose>
        <geometry>
            <capsule>
                <length>0.22</length>
                <radius>0.01</radius>
            </capsule>
        </geometry>
        
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="stick1">
        <pose>0 .85 10.2 0 .2 0</pose> 
        <geometry>
            <capsule>
                <length>0.22</length>
                <radius>0.01</radius>
            </capsule>
        </geometry>
        <material>
            <diffuse>1 .84 0.6 1</diffuse>
        </material>
    </visual>

    </link>

    <link name="stick2">
    <collision name="stick2">
        <pose>0 .1 10.2 2 0 2</pose>
        <geometry>
            <capsule>
                <length>0.22</length>
                <radius>0.01</radius>
            </capsule>
        </geometry>
        <surface>
            <bounce>
                <restitution_coefficient>0.5</restitution_coefficient>
            </bounce>
            <contact>
                <elastic_modulus>.48</elastic_modulus>
            </contact>
        </surface>
        <drake:proximity_properties>
          <drake:point_contact_stiffness>
            {stiffness}
          </drake:point_contact_stiffness>
          <drake:hunt_crossley_dissipation>
            {dissipation}
          </drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
    </collision>
    <visual name="stick2">
        <pose>0 .1 10.2 2 0 2</pose>
        <geometry>
            <capsule>
                <length>0.22</length>
                <radius>0.01</radius>
            </capsule>
        </geometry>
        <material>
            <diffuse>1 .84 0.6 1</diffuse>
        </material>
    </visual>
    <inertial>
        <pose>0 .1 10.2 2 0 2</pose>
        <mass>0.3</mass>
        <inertia>
            <ixx>{1/12 * 0.22**2 }</ixx>
            <iyy>{1/12 * 0.22**2 }</iyy>
            <izz>{1/2 * 0.01**2}</izz>
        </inertia>

    </inertial>
    </link>
    </model>
</sdf>
"""

robot_directives = """
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: allegro
    file: package://drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf
"""

meshcat.Delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=0.001
    )

parser = Parser(plant)
parser.AddModelsFromString(robot_directives, ".dmd.yaml")
instance = parser.AddModelsFromString(drum_kit, ".sdf")
instance = parser.AddModelsFromString(drum_sticks, ".sdf")
# instance = parser.AddModelsFromUrl(
#     "package://drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf"
# )

plant.Finalize()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()

context = diagram.CreateDefaultContext()

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0 if running_as_notebook else 0.1)

# visualizer.Run(loop_once=not running_as_notebook)



INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/90397c33cab9b7234d94c1018f2755bb9989c5a8.tar.gz
/Library/Developer/CommandLineTools/usr/bin/python3: can't open file '/Users/stvrh/Documents/MIT/Fall23/6.4210': [Errno 2] No such file or directory


RuntimeError: PackageMap: when downloading 'drake_models', the downloader experienced an error: returncode == 512

In [5]:
# stiffness=20000
# dissipation=2

# drum_kit = f"""
# <?xml version="1.0" ?>
# <sdf version="1.7">
#     <model name="drumkit">
#     <static>true</static>
#     <link name="snare">
#     <collision name="snare">
#         <pose>.1 .1 .4 0 .14 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.18</length>
#                 <radius>0.2</radius>
#             </cylinder>
#         </geometry>
#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="snare">
#         <pose>.1 .1 .4 0 .14 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.18</length>
#                 <radius>0.2</radius>
#             </cylinder>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>
#     <link name="tom1">
#     <collision name="tom1">
#         <pose>-.2 .3 .6 0 .55 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.1</length>
#                 <radius>0.2</radius>
#             </cylinder>
#         </geometry>
#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="tom1">
#         <pose>-.2 .3 .6 0 .55 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.15</length>
#                 <radius>0.14</radius>
#             </cylinder>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>

#     <link name="tom2">
#     <collision name="tom2">
#         <pose>-.2 .3 .6 0 .55 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.1</length>
#                 <radius>0.2</radius>
#             </cylinder>
#         </geometry>
#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="tom2">
#         <pose>-.2 .62 .6 0 .55 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.15</length>
#                 <radius>0.16</radius>
#             </cylinder>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>

#     <link name="tom3">
#     <collision name="tom3">
#         <pose>.1 .9 .3 0 0 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.4</length>
#                 <radius>0.2</radius>
#             </cylinder>
#         </geometry>
#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="tom3">
#         <pose>.1 .9 .3 0 0 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.4</length>
#                 <radius>0.2</radius>
#             </cylinder>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>

#     <link name="hihat">
#     <collision name="hihat">
#         <pose>-.1 -.1 .6 0 0 0</pose>
#         <geometry>
#             <ellipsoid>
#                 <radii>.2 .2 .02</radii>
#             </ellipsoid>
#         </geometry>

#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="hihat">
#         <pose>.1 -.25 .6 0 0 0</pose>
#         <geometry>
#             <ellipsoid>
#                 <radii>.2 .2 .02</radii>
#             </ellipsoid>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>

#     <link name="crash">
#     <collision name="crash">
#          <pose>-.4 .1 .85 0 .3 0</pose>
#         <geometry>
#             <ellipsoid>
#                 <radii>.26 .26 .005</radii>
#             </ellipsoid>
#         </geometry>

#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="crash">
#         <pose>-.4 .1 .85 0 .3 0</pose>
#         <geometry>
#             <ellipsoid>
#                 <radii>.26 .26 .005</radii>
#             </ellipsoid>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>

#     <link name="ride">
#     <collision name="ride">
#         <pose>-.25 1.1 .65 .3 .3 0</pose>
#         <geometry>
#             <ellipsoid>
#                 <radii>.3 .3 .005</radii>
#             </ellipsoid>
#         </geometry>

#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="ride">
#         <pose>-.25 1.1 .65 .3 .3 0</pose>
#         <geometry>
#             <ellipsoid>
#                 <radii>.3 .3 .005</radii>
#             </ellipsoid>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>

#     <link name="kick">
#     <collision name="kick">
#         <pose>-.3 .47 .25 0 1.57 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.4</length>
#                 <radius>0.25</radius>
#             </cylinder>
#         </geometry>

#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="kick">
#         <pose>-.3 .47 .25 0 1.57 0</pose>
#         <geometry>
#             <cylinder>
#                 <length>.4</length>
#                 <radius>0.25</radius>
#             </cylinder>
#         </geometry>
#         <material>
#             <diffuse>1 0 0 1</diffuse>
#         </material>
#     </visual>
#     </link>

#     </model>
#     </sdf>
# """


# drum_sticks  = f"""
# <?xml version="1.0" ?>
# <sdf version="1.7">
#     <model name="sticks">
#     <link name="stick1">
#     <inertial>
#         <pose>0 .85 10.2 0 .2 0</pose>
#         <mass>0.3</mass>
#         <inertia>
#             <ixx>{1/12 * 0.22**2 }</ixx>
#             <iyy>{1/12 * 0.22**2 }</iyy>
#             <izz>{1/2 * 0.01**2}</izz>
#         </inertia>
#         </inertial>
#     <collision name="stick1">
#         <pose>0 .85 10.2 0 .2 0</pose>
#         <geometry>
#             <capsule>
#                 <length>0.22</length>
#                 <radius>0.01</radius>
#             </capsule>
#         </geometry>
        
#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="stick1">
#         <pose>0 .85 10.2 0 .2 0</pose> 
#         <geometry>
#             <capsule>
#                 <length>0.22</length>
#                 <radius>0.01</radius>
#             </capsule>
#         </geometry>
#         <material>
#             <diffuse>1 .84 0.6 1</diffuse>
#         </material>
#     </visual>

#     </link>

#     <link name="stick2">
#     <collision name="stick2">
#         <pose>0 .1 10.2 2 0 2</pose>
#         <geometry>
#             <capsule>
#                 <length>0.22</length>
#                 <radius>0.01</radius>
#             </capsule>
#         </geometry>
#         <surface>
#             <bounce>
#                 <restitution_coefficient>0.5</restitution_coefficient>
#             </bounce>
#             <contact>
#                 <elastic_modulus>.48</elastic_modulus>
#             </contact>
#         </surface>
#         <drake:proximity_properties>
#           <drake:point_contact_stiffness>
#             {stiffness}
#           </drake:point_contact_stiffness>
#           <drake:hunt_crossley_dissipation>
#             {dissipation}
#           </drake:hunt_crossley_dissipation>
#         </drake:proximity_properties>
#     </collision>
#     <visual name="stick2">
#         <pose>0 .1 10.2 2 0 2</pose>
#         <geometry>
#             <capsule>
#                 <length>0.22</length>
#                 <radius>0.01</radius>
#             </capsule>
#         </geometry>
#         <material>
#             <diffuse>1 .84 0.6 1</diffuse>
#         </material>
#     </visual>
#     <inertial>
#         <pose>0 .1 10.2 2 0 2</pose>
#         <mass>0.3</mass>
#         <inertia>
#             <ixx>{1/12 * 0.22**2 }</ixx>
#             <iyy>{1/12 * 0.22**2 }</iyy>
#             <izz>{1/2 * 0.01**2}</izz>
#         </inertia>

#     </inertial>
#     </link>
#     </model>
# </sdf>
# """

# meshcat.Delete()

# builder = DiagramBuilder()
# plant, scene_graph = AddMultibodyPlantSceneGraph(
#         builder, time_step=0.00001
#     )

# parser = Parser(plant)
# instance = parser.AddModelsFromString(drum_kit, ".sdf")
# instance = parser.AddModelsFromString(drum_sticks, ".sdf")
# # instance = parser.AddModelsFromUrl(
# #     "package://drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf"
# # )

# plant.Finalize()
# visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# diagram = builder.Build()

# context = diagram.CreateDefaultContext()

# simulator = Simulator(diagram, context)
# simulator.set_target_realtime_rate(1.0)
# simulator.AdvanceTo(5.0 if running_as_notebook else 0.1);

# # visualizer.Run(loop_once=not running_as_notebook)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9b055803-8855-4ac7-95ae-1ed8fd517a9b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>